We attempted three models for this hackathon. In the first attempt, we use relu and max values to determine if the input has geiger values above the threshold. we had unsupported operators, and even after removing them had difficulty. same situation with 2nd and 3rd models until we hard coded the logic in Orion in the 3rd model (see the repo).

These are all the models we tried

In [ ]:
!pip install torch
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 77.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.onnx

class MotemaModel(nn.Module):
    def __init__(self, threshold):
        super(MotemaModel, self).__init__()
        self.threshold = threshold

    def forward(self, inputs):
        # Apply ReLU activation
        relu_output = nn.functional.relu(inputs)

        # Find the maximum value in the ReLU output
        max_value, _ = torch.max(relu_output, dim=-1)

        # Check if the maximum value is above threshold
        above_threshold = (max_value > self.threshold).any()

        return above_threshold

# Create an instance of the model
model = MotemaModel(threshold=0.5)

# Example input tensor
input_tensor = torch.tensor([[-1.0, 2.0, -0.5], [0.1, 0.2, 0.3]])

# Export the model to ONNX
output_path = "motema.onnx"
input_names = ["input"]
output_names = ["output"]

torch.onnx.export(
    model,
    input_tensor,
    output_path,
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=input_names,
    output_names=output_names,
    dynamic_axes={'input': {0: 'batch_size'},
                  'output': {0: 'batch_size'}}
)

print(f"Model exported to {output_path}")

Model exported to motema.onnx


In [ ]:
import onnx
from onnx import helper

def remove_cast_nodes(model):
    # Create a new graph with the same name and initializers
    new_graph = helper.make_graph(
        nodes=[],
        name=model.graph.name,
        inputs=model.graph.input,
        outputs=model.graph.output,
        initializer=model.graph.initializer
    )

    # Iterate through the original graph nodes
    for node in model.graph.node:
        # Skip Cast nodes
        if node.op_type == 'Cast':
            continue

        # Create a dictionary of attributes
        attributes = {attr.name: helper.get_attribute_value(attr) for attr in node.attribute}

        # Create a new node with the same attributes
        new_node = helper.make_node(
            op_type=node.op_type,
            inputs=[],
            outputs=node.output,
            name=node.name,
            **attributes
        )

        # Update the input connections for the new node
        for input_name in node.input:
            # Check if the input is from a Cast node
            if any(input_name in cast_node.output for cast_node in model.graph.node if cast_node.op_type == 'Cast'):
                # Find the input node for the Cast node
                cast_input_node = next(n for n in model.graph.node if input_name in n.output)
                new_node.input.extend(cast_input_node.input)
            else:
                new_node.input.append(input_name)

        # Add the new node to the new graph
        new_graph.node.append(new_node)

    # Create a new ONNX model with the updated graph
    new_model = helper.make_model(new_graph)

    return new_model

# Load the existing ONNX model
model = onnx.load('motema.onnx')

# Remove Cast nodes and create a new model
updated_model = remove_cast_nodes(model)

# Save the updated ONNX model
onnx.save(updated_model, 'new_motema.onnx')

DARNNNN! Even after removing the unsupported operations, I got an issue in Sierra that I don't have time to debug. Let's just move foward with a single layer Relu model that activates any time a value above zero is read. Hacky business.

In [ ]:
import torch
import torch.nn as nn
import torch.onnx

class ThresholdReLU(nn.Module):
    def __init__(self):
        super(ThresholdReLU, self).__init__()
        self.threshold = 10  # Hard-coded threshold value

    def forward(self, x):
        return torch.clamp(x, min=0, max=None) - torch.clamp(x - self.threshold, min=0, max=None) + self.threshold

# Example usage
input_tensor = torch.randn(3, 4, 5)  # Arbitrary dimension input tensor
threshold_relu = ThresholdReLU()
output_tensor = threshold_relu(input_tensor)
print(output_tensor)

# Export to ONNX
dummy_input = torch.randn(1, 4, 5)  # Dummy input tensor for tracing
onnx_path = "threshold_relu.onnx"
torch.onnx.export(threshold_relu, dummy_input, onnx_path, input_names=["input"], output_names=["output"])
print(f"ONNX model exported to {onnx_path}")

tensor([[[10.0000, 10.0000, 10.6845, 11.3676, 10.0000],
         [10.0000, 10.0000, 10.0000, 10.0000, 10.9450],
         [10.8528, 10.5069, 10.0000, 10.0000, 10.0000],
         [10.0000, 10.0000, 10.0000, 12.7550, 10.5501]],

        [[10.7883, 10.0000, 10.0000, 10.6433, 10.5456],
         [11.7491, 10.6858, 10.1643, 11.2402, 11.0839],
         [10.0000, 11.6770, 10.4200, 10.1146, 10.0000],
         [10.3730, 11.1745, 10.0624, 11.0636, 10.0000]],

        [[10.2130, 10.6126, 10.4162, 10.0000, 10.0000],
         [10.4675, 10.0000, 10.0000, 10.3210, 10.0000],
         [11.0580, 10.8105, 11.0548, 10.0000, 10.0000],
         [10.3731, 11.4597, 10.0000, 11.0546, 10.0000]]])
ONNX model exported to threshold_relu.onnx


Drat. Soiled again. We can't use clamp. YOLO SINGLE LAYER RELU

In [ ]:
import torch
import torch.nn as nn
import torch.onnx

class ThresholdReLU(nn.Module):
    def __init__(self, threshold=10):
        super(ThresholdReLU, self).__init__()
        self.threshold = threshold

    def forward(self, x):
        return torch.max(x, torch.zeros_like(x)) - torch.max(x - self.threshold, torch.zeros_like(x)) + self.threshold

# Example usage
input_tensor = torch.tensor([-5.0, 2.0, 8.0, -3.0, 12.0, 0.0, 7.0, -1.0, 10.0, 6.0])
threshold_relu = ThresholdReLU(threshold=10)
output_tensor = threshold_relu(input_tensor)
print(output_tensor)

# Export to ONNX
dummy_input = torch.randn(10)  # Dummy input tensor for tracing
onnx_path = "threshold_relu_two.onnx"
torch.onnx.export(threshold_relu, dummy_input, onnx_path, input_names=["input"], output_names=["output"])
print(f"ONNX model exported to {onnx_path}")

tensor([10., 12., 18., 10., 20., 10., 17., 10., 20., 16.])
ONNX model exported to threshold_relu_two.onnx
